In [3]:
import pandas as pd
import numpy as np

codeA = pd.read_csv('data/Train/tc_codea_코드A.csv')
codeB = pd.read_csv('data/Train/tc_codeb_코드B.csv')
code_area = pd.read_csv('data/Train/tc_sgg_시군구코드.csv')

# 라벨링 데이터

# 그 뭘 했었는지 - 방문장소 , 여기서 방문한 장소의 타입을 찾을 수 있음
visit_area_info = pd.read_csv('data/Train/tn_visit_area_info_방문지정보_merged.csv')
traveler_master = pd.read_csv('data/Train/tn_traveller_master_여행객 Master_merged.csv')

# 돈은 얼마나 쓰는지 - 지출내역
adv_consume_his_act = pd.read_csv('data/Train/tn_activity_consume_his_활동소비내역_t.csv')

# 뭘 타고는지 - 이동수단소비내역
move_consume_his = pd.read_csv('data/Train/tn_mvmn_consume_his_이동수단소비내역_merged.csv')

companion_info = pd.read_csv('data/Train/tn_companioninfo동반자정보_t.csv')

# 여행 목적 - 여행 페르소나
travel = pd.read_csv('./data/Train/tn_travel_여행_merged.csv')

# TODO: 감정 (무드)를 추출해보기 위한 데이터 추후 추가 필요

In [4]:
valid_area_type_codes = list(range(1, 9))
visit_area_info = visit_area_info[ (visit_area_info['VISIT_AREA_TYPE_CD'].isin(valid_area_type_codes))]
print(f"데이터 크기: {len(visit_area_info)}")

데이터 크기: 36646


In [5]:
visit_area_info = visit_area_info.reset_index(drop = True)

In [6]:
visit_area_info['VISIT_AREA_TYPE_CD'].unique()

array([2, 1, 6, 4, 3, 7, 5, 8])

In [7]:
visit_area_info.dropna(subset = ['LOTNO_ADDR'], inplace = True)
visit_area_info = visit_area_info.reset_index(drop = True)

In [8]:
sido = []
gungu = []
for i in range(len(visit_area_info['LOTNO_ADDR'])):
    sido.append(visit_area_info['LOTNO_ADDR'][i].split(' ')[0])
    gungu.append(visit_area_info['LOTNO_ADDR'][i].split(' ')[1])

In [9]:
visit_area_info['SIDO'] = sido
visit_area_info['GUNGU'] = gungu

In [10]:
visit_area_info['SIDO'].value_counts()

SIDO
제주특별자치도    8578
경기         3506
서울         2507
경북         2432
전남         2286
전북         1947
충남         1914
강원특별자치도    1870
부산         1758
경남         1459
강원         1308
충북         1213
인천         1205
대전         1171
대구          455
광주          322
울산          293
세종특별자치시     223
강원도           5
부산광역시         4
경상북도          4
전라남도          3
충청남도          3
경기도           3
전라북도          3
충청북도          2
경상남도          1
대전광역시         1
서울특별시         1
Name: count, dtype: int64

In [11]:
visit_area_info = visit_area_info[['TRAVEL_ID', 'VISIT_AREA_NM', 'SIDO', 'GUNGU', 'VISIT_AREA_TYPE_CD', 'DGSTFN',
                                'REVISIT_INTENTION', 'RCMDTN_INTENTION', 'RESIDENCE_TIME_MIN', 'REVISIT_YN']]

In [12]:
# TRAVEL_MISSION_CHECK의 첫번째 항목 가져오기
travel_list = []
for i in range(len(travel)):
    value = int(travel['TRAVEL_MISSION_CHECK'][i].split(';')[0])
    travel_list.append(value)

travel['TRAVEL_MISSION_PRIORITY'] = travel_list

In [13]:
travel = travel[['TRAVEL_ID', 'TRAVELER_ID', 'TRAVEL_MISSION_PRIORITY']]
travel.isna().sum().sum()

0

In [14]:
traveler_master = traveler_master[['TRAVELER_ID', 'GENDER', 'AGE_GRP', 'INCOME', 'TRAVEL_STYL_1', 
                                     'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 
                                     'TRAVEL_STYL_6', 'TRAVEL_STYL_7','TRAVEL_STYL_8', 
                                      'TRAVEL_MOTIVE_1', 'TRAVEL_NUM', 'TRAVEL_COMPANIONS_NUM' ]]
traveler_master.isna().sum().sum()

0

In [15]:
valid_codes = list(range(1, 8))  # 1부터 7까지의 리스트
adv_consume_his_act = adv_consume_his_act[
    (adv_consume_his_act['ACTIVITY_TYPE_CD'].isin(valid_codes))
]
print(f"결측치 제거 전 데이터 크기: {len(adv_consume_his_act)}")


결측치 제거 전 데이터 크기: 70874


In [16]:
# 결측치 제거
adv_consume_his_act = adv_consume_his_act.dropna(subset=['STORE_NM'])
adv_consume_his_act = adv_consume_his_act[['TRAVEL_ID', 'ACTIVITY_TYPE_CD', 'STORE_NM']]

print(f"결측치 제거 후 데이터 크기: {len(adv_consume_his_act)}")
adv_consume_his_act.to_csv('svd_travel_cosume_act_results.csv', index=False)
print()

결측치 제거 후 데이터 크기: 70627



# 동반자 정보
  - 1182,TCR,"1",배우자,"","",N,999,"2022-09-22 17:34:50",
  - 1183,TCR,"2",자녀,"","",N,999,"2022-09-22 17:34:50",
  - 1184,TCR,"3",부모,"","",N,999,"2022-09-22 17:34:50",
  - 1185,TCR,"4",조부모,"","",N,999,"2022-09-22 17:34:50",
  - 1186,TCR,"5",형제/자매,"","",N,999,"2022-09-22 17:34:50",
  - 1187,TCR,"6",친인척,"","",N,999,"2022-09-22 17:34:50",
  - 1188,TCR,"7",친구,"","",N,999,"2022-09-22 17:34:50",
  - 1189,TCR,"8",연인,"","",N,999,"2022-09-22 17:34:50",
  - 1190,TCR,"9",동료,"","",N,999,"2022-09-22 17:34:50",
  - 1191,TCR,"10","친목 단체/모임(동호회, 종교단체 등)","","",N,999,"2022-09-22 17:34:50",
  - 1192,TCR,"11",기타,"","",N,999,"2022-09-22 17:34:50"

In [17]:
valid_companion_codes = list(range(1, 11)) # 1부터 10까지의 리스트
companion_info = companion_info[(companion_info['REL_CD'].isin(valid_companion_codes))]

companion_info = companion_info[['TRAVEL_ID', 'REL_CD']]

In [18]:
move_consume_his = move_consume_his[['TRAVEL_ID', 'MVMN_SE_NM']]

In [19]:
df = pd.merge(travel, traveler_master, left_on = 'TRAVELER_ID', right_on = 'TRAVELER_ID', how = 'inner')
df.to_csv('svd_travel_0_results.csv', index=False)

In [20]:
df = pd.merge(visit_area_info, df, left_on = 'TRAVEL_ID', right_on = 'TRAVEL_ID', how = 'right')
df.to_csv('svd_travel_1_results.csv', index=False)

In [21]:
df = pd.merge(companion_info, df, left_on = 'TRAVEL_ID', right_on = 'TRAVEL_ID', how = 'right')
df.to_csv('svd_travel_2_results.csv', index=False)

In [22]:
df = pd.merge(adv_consume_his_act, df, left_on = 'TRAVEL_ID', right_on = 'TRAVEL_ID', how = 'right')
df.to_csv('svd_travel_3_results.csv', index=False)

In [23]:
df = pd.merge(move_consume_his, df, left_on = 'TRAVEL_ID', right_on = 'TRAVEL_ID', how = 'right')

In [ ]:
df['RESIDENCE_TIME_MIN'] = df['RESIDENCE_TIME_MIN'].replace(0,60)

In [25]:
df['REVISIT_YN'] = df['REVISIT_YN'].replace("N",0)
df['REVISIT_YN'] = df['REVISIT_YN'].replace("Y",1)

In [26]:
df.dropna(subset = ['TRAVEL_STYL_1'], inplace = True)
df.reset_index(drop= True, inplace = True)

In [27]:
df.dropna(subset = ['TRAVEL_MOTIVE_1'], inplace = True)
df.reset_index(drop= True, inplace = True)

In [28]:
df.shape

(1928617, 30)

In [29]:
print(df)
# df.to_csv('svd_results.csv', index=False)
print(f"중복 행 개수: {df.duplicated().sum()}")
df = df.drop_duplicates()
print(f"중복 제거 후 데이터 크기: {df.shape}")
# df.to_csv('svd_results.csv', index=False)

         TRAVEL_ID        MVMN_SE_NM  ACTIVITY_TYPE_CD            STORE_NM  \
0        e_e000004               NaN               NaN                 NaN   
1        e_e000004               NaN               NaN                 NaN   
2        e_e000004               NaN               NaN                 NaN   
3        e_e000004               NaN               NaN                 NaN   
4        e_e000004               NaN               NaN                 NaN   
...            ...               ...               ...                 ...   
1928612  h_h006874  자가용(승용/승합/트럭 등등)               1.0          민경이네어등포해녀촌   
1928613  h_h006874  자가용(승용/승합/트럭 등등)               2.0  마이피기팬트리(하도리1091펜션)   
1928614  h_h006874  자가용(승용/승합/트럭 등등)               1.0                꼬스뗀뇨   
1928615  h_h006874  자가용(승용/승합/트럭 등등)               1.0          민경이네어등포해녀촌   
1928616  h_h006874  자가용(승용/승합/트럭 등등)               2.0  마이피기팬트리(하도리1091펜션)   

         REL_CD        VISIT_AREA_NM     SIDO GUNGU  VISIT_AREA

In [30]:
df.to_csv('svd_results.csv', index=False)

In [31]:
df.isna().sum().sum()
missing_percentage = (df.isna().sum().sum() / (df.shape[0] * df.shape[1])) * 100
print(f"결측치 비율: {missing_percentage:.2f}%")

결측치 비율: 0.65%


In [32]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

df_original = df.copy()
df1 = df_original.copy()
Train = pd.DataFrame(columns=list(df_original.columns))


In [33]:
# ## 2. Train-Test Split 수행

# %%
# 2.1 초기 데이터프레임 설정
df1 = df_original.copy()
Train = pd.DataFrame(columns=list(df_original.columns))


In [34]:
# 최적화된 버전 + 상세 진행도 표시
from tqdm import tqdm
import pandas as pd
import numpy as np

# 초기 설정
df1 = df_original.copy()
train_travel_ids = set()  # 이미 Train에 추가된 travel_id 집합
Train = pd.DataFrame(columns=list(df_original.columns))

# 진행 상황 표시 준비
total_places = len(df_original['VISIT_AREA_NM'].unique())
print(f"총 관광지 수: {total_places}개")

# 관광지별 사용자 인덱스 미리 계산 (진행도 표시)
print("관광지별 사용자 데이터 준비 중...")
place_to_users = {}
for place in tqdm(df_original['VISIT_AREA_NM'].unique(), desc="관광지 맵핑"):
    place_to_users[place] = set(df_original[df_original['VISIT_AREA_NM'] == place]['TRAVEL_ID'])

# 각 관광지마다 최소 1명의 유저를 Train set에 추가 (상세 진행도 추가)
print("\n각 관광지별 대표 사용자 추가 중...")
added_places = 0
total_users_added = 0

for place in tqdm(list(df_original['VISIT_AREA_NM'].unique()), desc="관광지 처리"):
    # 아직 Train에 추가되지 않은 이 관광지 방문자들
    available_users = place_to_users[place] - train_travel_ids
    
    if not available_users:
        continue
    
    # 랜덤하게 한 사용자 선택
    np.random.seed(42)
    selected_user = np.random.choice(list(available_users))
    
    # 선택된 사용자의 모든 방문 데이터 추가
    user_data = df1[df1['TRAVEL_ID'] == selected_user]
    Train = pd.concat([Train, user_data], ignore_index=True)
    
    # 추가된 사용자 ID 기록
    train_travel_ids.add(selected_user)
    
    # df1에서 해당 사용자 데이터 제거
    df1 = df1[df1['TRAVEL_ID'] != selected_user]
    
    # 상세 진행 상황 업데이트
    added_places += 1
    total_users_added += 1
    
    # 매 100개 관광지마다 상황 표시
    if added_places % 100 == 0:
        print(f"진행 상황: {added_places}/{total_places} 관광지 처리 ({added_places/total_places*100:.1f}%)")
        print(f"현재까지 {total_users_added}명의 사용자가 Train set에 추가됨")
        print(f"Train set 크기: {len(Train)}, Test set 크기: {len(df1)}")

# 최종 결과 표시
print(f"\n완료! 총 {added_places}개 관광지의 {total_users_added}명 사용자가 Train set에 추가됨")
print(f"Train set 크기: {len(Train)}행 ({len(Train)/len(df_original)*100:.1f}%)")
print(f"남은 Test set 크기: {len(df1)}행 ({len(df1)/len(df_original)*100:.1f}%)")

총 관광지 수: 11468개
관광지별 사용자 데이터 준비 중...


관광지 맵핑:   0%|          | 29/11468 [00:00<05:09, 37.01it/s]


KeyboardInterrupt: 

In [ ]:
# ### 2.3 남은 유저들을 80:20 비율로 분할

# %%
# 80:20 비율 맞추기
while len(df1)/len(df_original) > 0.2:
    np.random.seed(42)
    if len(df1) == 0:
        break
    random_number = np.random.randint(len(df1))
    df_id = df1.iloc[[random_number]]
    index = df_id.iloc[0,0]
    df3 = df1[df1['TRAVEL_ID'] == index]
    df1 = pd.merge(df3, df1, how='outer', indicator=True)
    df1 = df1.query('_merge =="right_only"').drop(columns=['_merge'])
    Train = pd.concat([Train, df3], ignore_index=True)


In [ ]:
# ### 2.4 분할 결과 확인

# %%
# 분할 결과 검증
print(f"Train set 크기: {len(Train)} ({len(Train)/len(df_original)*100:.1f}%)")
print(f"Test set 크기: {len(df1)} ({len(df1)/len(df_original)*100:.1f}%)")

# 관광지 포함 여부 확인
train_places = set(Train['VISIT_AREA_NM'].unique())
test_places = set(df1['VISIT_AREA_NM'].unique())
print(f"Train set 관광지 수: {len(train_places)}")
print(f"Test set 관광지 수: {len(test_places)}")
print(f"Train set에만 있는 관광지 수: {len(train_places - test_places)}")
print(f"Test set에만 있는 관광지 수: {len(test_places - train_places)}")

# 모든 관광지가 train set에 포함되었는지 확인
all_places = set(df_original['VISIT_AREA_NM'].unique())
missing_places = all_places - train_places
if len(missing_places) > 0:
    print(f"주의: Train set에 없는 관광지가 {len(missing_places)}개 있습니다.")
else:
    print("성공: 모든 관광지가 Train set에 포함되어 있습니다.")


Train set 크기: 745214 (80.0%)
Test set 크기: 186295 (20.0%)
Train set 관광지 수: 11467
Test set 관광지 수: 1855
Train set에만 있는 관광지 수: 9613
Test set에만 있는 관광지 수: 1
주의: Train set에 없는 관광지가 1개 있습니다.


In [ ]:
# ## 3. 관광지별 통계 계산 (기본 + 확장 특성)

# %%
# 3.1 통계 계산을 위한 기본 컬럼 정의
basic_stat_columns = [
    'RESIDENCE_TIME_MIN',      # 체류시간
    'RCMDTN_INTENTION',        # 추천의향
    'REVISIT_YN',              # 재방문여부
    'TRAVEL_COMPANIONS_NUM',   # 동반자 수
    'REVISIT_INTENTION',       # 재방문의향
    'DGSTFN'                   # 만족도
]

# %% [markdown]
# ### 3.2 기본 통계값 계산

# %%
# 새로운 데이터프레임 생성
stat_columns = [f'{col}_mean' for col in basic_stat_columns]
new_train = pd.DataFrame(columns=list(Train.columns) + stat_columns)

# 각 관광지별 기본 통계 계산
for i in tqdm(list(Train['VISIT_AREA_NM'].unique())):
    df2 = Train[Train['VISIT_AREA_NM'] == i]
    
    # 기본 통계값 계산
    for j in basic_stat_columns:
        mean_value = df2[j].mean()
        df2[f'{j}_mean'] = mean_value
    
    # 새로운 데이터프레임에 추가
    new_train = pd.concat([new_train, df2], axis=0)

# 정렬 및 인덱스 재설정
new_train.sort_values(by=['TRAVEL_ID'], axis=0, inplace=True)
new_train.reset_index(drop=True, inplace=True)


100%|██████████| 11467/11467 [08:10<00:00, 23.36it/s]


In [ ]:
# ### 3.3 확장된 특성 통계 계산 - 방문객 특성별 분포

# %%
# 방문지별 통계 데이터프레임 생성
place_stats = pd.DataFrame()
place_stats['VISIT_AREA_NM'] = Train['VISIT_AREA_NM'].unique()

# %% [markdown]
# #### 3.3.1 성별, 연령대, 소득 분포

# %%
# 성별 분포
gender_pivot = pd.pivot_table(
    Train, 
    values='TRAVEL_ID', 
    index='VISIT_AREA_NM', 
    columns='GENDER', 
    aggfunc='count',
    fill_value=0
)
gender_pivot = gender_pivot.div(gender_pivot.sum(axis=1), axis=0)
gender_pivot.columns = [f'GENDER_RATIO_{col}' for col in gender_pivot.columns]
place_stats = place_stats.merge(gender_pivot.reset_index(), on='VISIT_AREA_NM', how='left')

# 연령대 분포
age_pivot = pd.pivot_table(
    Train, 
    values='TRAVEL_ID', 
    index='VISIT_AREA_NM', 
    columns='AGE_GRP', 
    aggfunc='count',
    fill_value=0
)
age_pivot = age_pivot.div(age_pivot.sum(axis=1), axis=0)
age_pivot.columns = [f'AGE_RATIO_{col}' for col in age_pivot.columns]
place_stats = place_stats.merge(age_pivot.reset_index(), on='VISIT_AREA_NM', how='left')

# 소득 분포
income_pivot = pd.pivot_table(
    Train, 
    values='TRAVEL_ID', 
    index='VISIT_AREA_NM', 
    columns='INCOME', 
    aggfunc='count',
    fill_value=0
)
income_pivot = income_pivot.div(income_pivot.sum(axis=1), axis=0)
income_pivot.columns = [f'INCOME_RATIO_{col}' for col in income_pivot.columns]
place_stats = place_stats.merge(income_pivot.reset_index(), on='VISIT_AREA_NM', how='left')


NameError: name 'pd' is not defined

In [ ]:
# #### 3.3.2 여행 목적 및 여행 스타일 분포

# %%
# 여행 목적(미션) 분포
mission_pivot = pd.pivot_table(
    Train, 
    values='TRAVEL_ID', 
    index='VISIT_AREA_NM', 
    columns='TRAVEL_MISSION_PRIORITY', 
    aggfunc='count',
    fill_value=0
)
mission_pivot = mission_pivot.div(mission_pivot.sum(axis=1), axis=0)
mission_pivot.columns = [f'MISSION_RATIO_{col}' for col in mission_pivot.columns]
place_stats = place_stats.merge(mission_pivot.reset_index(), on='VISIT_AREA_NM', how='left')

# 여행 동기 분포
motive_pivot = pd.pivot_table(
    Train, 
    values='TRAVEL_ID', 
    index='VISIT_AREA_NM', 
    columns='TRAVEL_MOTIVE_1', 
    aggfunc='count',
    fill_value=0
)
motive_pivot = motive_pivot.div(motive_pivot.sum(axis=1), axis=0)
motive_pivot.columns = [f'MOTIVE_RATIO_{col}' for col in motive_pivot.columns]
place_stats = place_stats.merge(motive_pivot.reset_index(), on='VISIT_AREA_NM', how='left')


In [ ]:
# #### 3.3.3 여행 스타일 평균값 계산

# %%
# 여행 스타일 평균값
for i in range(1, 9):
    style_col = f'TRAVEL_STYL_{i}'
    style_means = Train.groupby('VISIT_AREA_NM')[style_col].mean()
    style_means.name = f'{style_col}_mean'
    place_stats = place_stats.merge(style_means.reset_index(), on='VISIT_AREA_NM', how='left')

# %% [markdown]
# #### 3.3.4 관광지 방문 빈도 및 유형 통계

# %%
# 방문 빈도
visit_counts = Train.groupby('VISIT_AREA_NM').size().rename('VISIT_FREQUENCY')
place_stats = place_stats.merge(visit_counts.reset_index(), on='VISIT_AREA_NM', how='left')

# 관광지 유형 분포
area_type_pivot = pd.pivot_table(
    Train, 
    values='TRAVEL_ID', 
    index='VISIT_AREA_NM', 
    columns='VISIT_AREA_TYPE_CD', 
    aggfunc='count',
    fill_value=0
)
area_type_pivot = area_type_pivot.div(area_type_pivot.sum(axis=1), axis=0)
area_type_pivot.columns = [f'AREA_TYPE_RATIO_{col}' for col in area_type_pivot.columns]
place_stats = place_stats.merge(area_type_pivot.reset_index(), on='VISIT_AREA_NM', how='left')


In [ ]:
# #### 3.3.5 지역 정보 통계

# %%
# 시도별 방문 비율
sido_pivot = pd.pivot_table(
    Train, 
    values='TRAVEL_ID', 
    index='VISIT_AREA_NM', 
    columns='SIDO', 
    aggfunc='count',
    fill_value=0
)
sido_pivot = sido_pivot.div(sido_pivot.sum(axis=1), axis=0)
sido_pivot.columns = [f'SIDO_RATIO_{col}' for col in sido_pivot.columns]
place_stats = place_stats.merge(sido_pivot.reset_index(), on='VISIT_AREA_NM', how='left')


In [ ]:
# #### 3.3.6 동반자 유형 통계 (REL_CD가 있는 경우)

# %%
# 동반자 데이터가 있는 경우에만 실행
if 'REL_CD' in Train.columns:
    rel_pivot = pd.pivot_table(
        Train, 
        values='TRAVEL_ID', 
        index='VISIT_AREA_NM', 
        columns='REL_CD', 
        aggfunc='count',
        fill_value=0
    )
    rel_pivot = rel_pivot.div(rel_pivot.sum(axis=1), axis=0)
    rel_pivot.columns = [f'REL_CD_RATIO_{col}' for col in rel_pivot.columns]
    place_stats = place_stats.merge(rel_pivot.reset_index(), on='VISIT_AREA_NM', how='left')


In [ ]:
# #### 3.3.7 지출 패턴 통계 (소비 내역 데이터가 있는 경우)

# %%
# 소비 내역 데이터가 있는 경우에만 실행
if 'ACTIVITY_TYPE_CD' in Train.columns:
    activity_pivot = pd.pivot_table(
        Train, 
        values='TRAVEL_ID', 
        index='VISIT_AREA_NM', 
        columns='ACTIVITY_TYPE_CD', 
        aggfunc='count',
        fill_value=0
    )
    activity_pivot = activity_pivot.div(activity_pivot.sum(axis=1), axis=0)
    activity_pivot.columns = [f'ACTIVITY_RATIO_{col}' for col in activity_pivot.columns]
    place_stats = place_stats.merge(activity_pivot.reset_index(), on='VISIT_AREA_NM', how='left')


In [ ]:
# #### 3.3.8 이동수단 선호도 통계 (이동수단 데이터가 있는 경우)

# %%
# 이동수단 데이터가 있는 경우에만 실행
if 'MVMN_SE_NM' in Train.columns:
    mvmn_pivot = pd.pivot_table(
        Train, 
        values='TRAVEL_ID', 
        index='VISIT_AREA_NM', 
        columns='MVMN_SE_NM', 
        aggfunc='count',
        fill_value=0
    )
    mvmn_pivot = mvmn_pivot.div(mvmn_pivot.sum(axis=1), axis=0)
    mvmn_pivot.columns = [f'TRANSPORT_RATIO_{col}' for col in mvmn_pivot.columns]
    place_stats = place_stats.merge(mvmn_pivot.reset_index(), on='VISIT_AREA_NM', how='left')


In [ ]:
# #### 3.3.9 통계 결과 확인 및 결측치 처리

# %%
# 결측치 0으로 대체
place_stats.fillna(0, inplace=True)

# 통계 결과 확인
print(f"계산된 관광지별 통계 수: {len(place_stats)}")
print(f"통계 특성 수: {len(place_stats.columns) - 1}")  # VISIT_AREA_NM 컬럼 제외
place_stats.head()

계산된 관광지별 통계 수: 11467
통계 특성 수: 125


,VISIT_AREA_NM,GENDER_RATIO_남,GENDER_RATIO_여,AGE_RATIO_20,AGE_RATIO_30,AGE_RATIO_40,AGE_RATIO_50,AGE_RATIO_60,INCOME_RATIO_1,INCOME_RATIO_2,INCOME_RATIO_3,INCOME_RATIO_4,INCOME_RATIO_5,INCOME_RATIO_6,INCOME_RATIO_7,INCOME_RATIO_8,INCOME_RATIO_9,INCOME_RATIO_10,INCOME_RATIO_11,INCOME_RATIO_12,MISSION_RATIO_1,MISSION_RATIO_2,MISSION_RATIO_3,MISSION_RATIO_4,MISSION_RATIO_5,MISSION_RATIO_6,MISSION_RATIO_7,MISSION_RATIO_8,MISSION_RATIO_9,MISSION_RATIO_10,MISSION_RATIO_11,MISSION_RATIO_12,MISSION_RATIO_13,MISSION_RATIO_21,MISSION_RATIO_22,MISSION_RATIO_23,MISSION_RATIO_24,MISSION_RATIO_25,MISSION_RATIO_26,MISSION_RATIO_27,MISSION_RATIO_28,MOTIVE_RATIO_1,MOTIVE_RATIO_2,MOTIVE_RATIO_3,MOTIVE_RATIO_4,MOTIVE_RATIO_5,MOTIVE_RATIO_6,MOTIVE_RATIO_7,MOTIVE_RATIO_8,MOTIVE_RATIO_9,MOTIVE_RATIO_10,TRAVEL_STYL_1_mean,TRAVEL_STYL_2_mean,TRAVEL_STYL_3_mean,TRAVEL_STYL_4_mean,TRAVEL_STYL_5_mean,TRAVEL_STYL_6_mean,TRAVEL_STYL_7_mean,TRAVEL_STYL_8_mean,VISIT_FREQUENCY,AREA_TYPE_RATIO_1.0,AREA_TYPE_RATIO_2.0,AREA_TYPE_RATIO_3.0,AREA_TYPE_RATIO_4.0,AREA_TYPE_RATIO_5.0,AREA_TYPE_RATIO_6.0,AREA_TYPE_RATIO_7.0,AREA_TYPE_RATIO_8.0,SIDO_RATIO_강원,SIDO_RATIO_강원도,SIDO_RATIO_강원특별자치도,SIDO_RATIO_경기,SIDO_RATIO_경기도,SIDO_RATIO_경남,SIDO_RATIO_경북,SIDO_RATIO_경상남도,SIDO_RATIO_경상북도,SIDO_RATIO_광주,SIDO_RATIO_대구,SIDO_RATIO_대전,SIDO_RATIO_대전광역시,SIDO_RATIO_부산,SIDO_RATIO_부산광역시,SIDO_RATIO_서울,SIDO_RATIO_서울특별시,SIDO_RATIO_세종특별자치시,SIDO_RATIO_울산,SIDO_RATIO_인천,SIDO_RATIO_전남,SIDO_RATIO_전라남도,SIDO_RATIO_전라북도,SIDO_RATIO_전북,SIDO_RATIO_제주특별자치도,SIDO_RATIO_충남,SIDO_RATIO_충북,SIDO_RATIO_충청남도,SIDO_RATIO_충청북도,REL_CD_RATIO_1.0,REL_CD_RATIO_2.0,REL_CD_RATIO_3.0,REL_CD_RATIO_4.0,REL_CD_RATIO_5.0,REL_CD_RATIO_6.0,REL_CD_RATIO_7.0,REL_CD_RATIO_8.0,REL_CD_RATIO_9.0,REL_CD_RATIO_10.0,ACTIVITY_RATIO_1.0,ACTIVITY_RATIO_2.0,ACTIVITY_RATIO_3.0,ACTIVITY_RATIO_6.0,TRANSPORT_RATIO_KTX/SRT(고속열차),TRANSPORT_RATIO_관광버스,TRANSPORT_RATIO_기타,TRANSPORT_RATIO_렌터카(승용/승합/버스 등등),TRANSPORT_RATIO_배/선박,TRANSPORT_RATIO_버스 + 지하철,TRANSPORT_RATIO_새마을/무궁화열차,TRANSPORT_RATIO_시내/마을버스,TRANSPORT_RATIO_시외/고속버스,TRANSPORT_RATIO_자가용(승용/승합/트럭 등등),TRANSPORT_RATIO_자전거,TRANSPORT_RATIO_지하철,TRANSPORT_RATIO_캠핑카(자차 및 렌탈),TRANSPORT_RATIO_택시,TRANSPORT_RATIO_항공기
0,화성 관광열차 안내소 연무대 매표소,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.0,4.0,3.0,5.0,2.0,3.0,6.0,5.0,6.0,10,0.000000,1.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500000,0.500000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.600000,0.000000,0.400000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,0.000000,0.0
1,화성행궁,0.297357,0.702643,0.612335,0.224670,0.081498,0.000000,0.081498,0.070485,0.123348,0.211454,0.436123,0.085903,0.035242,0.037445,0.00000,0.0,0.0,0.0,0.0,0.134361,0.017621,0.574890,0.037445,0.0,0.013216,0.0,0.0,0.0,0.0,0.0,0.035242,0.0,0.011013,0.176211,0.0,0.0,0.0,0.0,0.0,0.0,0.314978,0.138767,0.182819,0.055066,0.033040,0.022026,0.136564,0.092511,0.024229,0.0,3.596916,3.185022,3.220264,3.535242,3.60793,4.299559,4.0,4.414097,454,0.008811,0.95815,0.0,0.0,0.0,0.0,0.0,0.03304,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.087533,0.135279,0.029178,0.0,0.034483,0.0,0.336870,0.376658,0.0,0.0,0.634361,0.165198,0.176211,0.024229,0.052863,0.0,0.000000,0.000000,0.0,0.196035,0.059471,0.077093,0.088106,0.251101,0.013216,0.143172,0.0,0.118943,0.0
2,수원 화성 북서적대,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.000000,0.000000,1.000000,0.000000,0.0,0.0000

In [ ]:
# ## 4. Test Set에 관광지별 통계 적용

# %%
# 4.1 기본 통계값 적용
test_with_stats = df1.copy()

# 기본 통계값 컬럼 추가
for col in basic_stat_columns:
    test_with_stats[f'{col}_mean'] = np.nan

In [ ]:
# 4.2 각 행에 해당 관광지의 통계값 적용
for idx, row in tqdm(test_with_stats.iterrows(), total=len(test_with_stats)):
    place = row['VISIT_AREA_NM']
    place_subset = Train[Train['VISIT_AREA_NM'] == place]
    
    if len(place_subset) > 0:  # 해당 관광지가 Train set에 있는 경우
        for col in basic_stat_columns:
            test_with_stats.loc[idx, f'{col}_mean'] = place_subset[col].mean()


100%|██████████| 186295/186295 [1:03:31<00:00, 48.88it/s] 


In [ ]:
# ### 4.3 확장된 특성 적용

# %%
# place_stats에서 필요한 열만 선택 (VISIT_AREA_NM과 통계 열만)
stats_cols = [col for col in place_stats.columns if col != 'VISIT_AREA_NM']

# test_with_stats와 place_stats 병합
test_with_extended_stats = test_with_stats.merge(
    place_stats, 
    on='VISIT_AREA_NM',
    how='left'
)

In [ ]:
# 결측치 확인
missing_stats = test_with_extended_stats[stats_cols].isna().sum()
if missing_stats.sum() > 0:
    print("결측치가 있는 특성:")
    print(missing_stats[missing_stats > 0])
    # 결측치 0으로 대체
    test_with_extended_stats.fillna(0, inplace=True)
else:
    print("모든 통계값이 성공적으로 적용되었습니다.")


결측치가 있는 특성:
GENDER_RATIO_남                  3229
GENDER_RATIO_여                  3229
AGE_RATIO_20                    3229
AGE_RATIO_30                    3229
AGE_RATIO_40                    3229
                                ... 
TRANSPORT_RATIO_자전거             3229
TRANSPORT_RATIO_지하철             3229
TRANSPORT_RATIO_캠핑카(자차 및 렌탈)    3229
TRANSPORT_RATIO_택시              3229
TRANSPORT_RATIO_항공기             3229
Length: 125, dtype: int64


In [ ]:
# ## 5. 최종 데이터셋 저장

# %%
# 데이터셋 저장
new_train.to_csv('관광지_추천시스템_Trainset_확장.csv', index=False)
test_with_extended_stats.to_csv('관광지_추천시스템_Testset_확장.csv', index=False)
place_stats.to_csv('관광지_통계정보_확장.csv', index=False)

print("확장된 특성을 포함한 데이터셋 저장 완료!")


확장된 특성을 포함한 데이터셋 저장 완료!


In [ ]:
# ## 6. 데이터 검증 및 정리

# %%
# 결측치 제거 및 최종 크기 확인
new_train.dropna(inplace=True)
new_train.reset_index(drop=True, inplace=True)

test_with_extended_stats.dropna(inplace=True)
test_with_extended_stats.reset_index(drop=True, inplace=True)

print(f"최종 Train set 크기: {new_train.shape}")
print(f"최종 Test set 크기: {test_with_extended_stats.shape}")
print(f"관광지별 통계 특성 수: {len(place_stats.columns)}")


최종 Train set 크기: (618694, 36)
최종 Test set 크기: (186295, 161)
관광지별 통계 특성 수: 126


In [ ]:
# ## 7. 방문 빈도가 낮은 관광지 필터링 (5회 미만 방문)

# %%
# 방문 빈도 확인
visit_counts = pd.DataFrame(new_train['VISIT_AREA_NM'].value_counts())
visit_counts.columns = ['count']  # 열 이름 지정
visit_counts['places'] = visit_counts.index

five_or_more_places = list(visit_counts[visit_counts['count'] >= 5]['places'])
print(f"5회 이상 방문한 관광지 수: {len(five_or_more_places)}")

# 필터링 적용
filtered_train = new_train.copy()
filtered_indices = []
for i in range(len(filtered_train)):
    if filtered_train.loc[i, 'VISIT_AREA_NM'] not in five_or_more_places:
        filtered_indices.append(i)
        
filtered_train = filtered_train.drop(filtered_indices, axis=0)
filtered_train.reset_index(drop=True, inplace=True)

print(f"필터링 전 Train set 크기: {new_train.shape}")
print(f"필터링 후 Train set 크기: {filtered_train.shape}")
print(f"제거된 행 수: {len(new_train) - len(filtered_train)}")

# 필터링된 데이터셋 저장
filtered_train.to_csv('관광지_추천시스템_Trainset_필터링_확장.csv', index=False)

print("Train-Test Split 및 확장 특성 계산 완료!")

5회 이상 방문한 관광지 수: 7658
필터링 전 Train set 크기: (618694, 36)
필터링 후 Train set 크기: (614294, 36)
제거된 행 수: 4400
Train-Test Split 및 확장 특성 계산 완료!


In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
import joblib
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore') # Suppress warning messages
pd.set_option('display.max_columns', None)

# Load the processed datasets from the first code file
print("Loading preprocessed datasets...")
filtered_train = pd.read_csv('관광지_추천시스템_Trainset_필터링_확장.csv')
test_with_extended_stats = pd.read_csv('관광지_추천시스템_Testset_확장.csv')
place_stats = pd.read_csv('관광지_통계정보_확장.csv')

# Check the data
print(f"Train set shape: {filtered_train.shape}")
print(f"Test set shape: {test_with_extended_stats.shape}")
print(f"Place stats shape: {place_stats.shape}")

# Prepare data for CatBoost model
# Remove columns that are not needed for prediction
print("Preparing data for CatBoost model...")
train_cols_to_drop = ['TRAVEL_ID', 'TRAVELER_ID', 'REVISIT_INTENTION', 
                     'RCMDTN_INTENTION', 'RESIDENCE_TIME_MIN', 'REVISIT_YN']
y_train = filtered_train['DGSTFN']
X_train = filtered_train.drop(['DGSTFN'] + train_cols_to_drop, axis=1, errors='ignore')

# Detect potential categorical columns and convert them to string type for CatBoost
# This ensures CatBoost treats them properly rather than trying to convert to float
categorical_features = []
print("Identifying and converting categorical features...")

# Examine each column to determine categorical vs numerical
for col in X_train.columns:
    # Check first few values to determine if column might be categorical
    sample_values = X_train[col].dropna().head(5)
    is_categorical = False
    
    # If column name suggests it's categorical
    if any(keyword in col.lower() for keyword in ['nm', 'cd', 'type', 'id', 'gender', 'age', 'sido', 'gungu', 'priority']):
        is_categorical = True
    # Or if values are non-numeric
    elif any(isinstance(val, str) for val in sample_values):
        is_categorical = True
    # Or if number of unique values is small (likely categorical)
    elif X_train[col].nunique() < 20:
        is_categorical = True
        
    if is_categorical:
        categorical_features.append(col)
        # Convert to string to ensure proper handling
        X_train[col] = X_train[col].astype(str)
        
print(f"Identified {len(categorical_features)} categorical features:")
print(categorical_features)
print(f"Total features: {X_train.shape[1]}")

# Train CatBoost model with optimal hyperparameters from the second file
print("Training CatBoost model...")
model = CatBoostRegressor(
    n_estimators=1150,
    max_depth=10,
    subsample=0.95,
    colsample_bylevel=0.95,
    cat_features=categorical_features,
    random_state=42,
    verbose=100  # Show progress every 100 iterations
)

try:
    print("Training CatBoost model...")
    model.fit(X_train, y_train)
except Exception as e:
    print(f"Error during model training: {str(e)}")
    
    # If training fails, let's try with more careful feature handling
    print("Retrying with more careful feature handling...")
    
    # Convert all columns to appropriate types
    for col in X_train.columns:
        if col in categorical_features:
            X_train[col] = X_train[col].astype(str)
        else:
            # For numeric columns, ensure they're float type and handle missing values
            try:
                X_train[col] = pd.to_numeric(X_train[col], errors='coerce')
                X_train[col] = X_train[col].fillna(X_train[col].mean())
            except:
                # If conversion fails, treat as categorical
                print(f"Converting {col} to categorical due to numeric conversion issue")
                categorical_features.append(col)
                X_train[col] = X_train[col].astype(str)
    
    # Try again with updated features
    model = CatBoostRegressor(
        n_estimators=1150,
        max_depth=10,
        subsample=0.95,
        colsample_bylevel=0.95,
        cat_features=categorical_features,
        random_state=42,
        verbose=100
    )
    
    model.fit(X_train, y_train)

# Save the model
print("Saving model...")
timestamp = time.strftime('%Y%m%d_%H%M%S')
model_path = f'catboost_tourism_recommendation_model_{timestamp}.pkl'
joblib.dump(model, model_path)
print(f"Model saved to {model_path}")

# Prepare test set data for recommendation generation
print("Generating recommendations for test set...")

# Extract user information
data = test_with_extended_stats[['TRAVEL_ID', 'SIDO', 'GUNGU', 'TRAVEL_MISSION_PRIORITY', 
                                'GENDER', 'AGE_GRP', 'INCOME',
                                'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4',
                                'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8',
                                'TRAVEL_MOTIVE_1', 'TRAVEL_NUM', 'TRAVEL_COMPANIONS_NUM']]

# Create a DataFrame with user information and the list of regions they've visited
print("Processing user region information...")
data1 = pd.DataFrame(columns=['TRAVEL_ID', 'TRAVEL_MISSION_PRIORITY', 'GENDER', 'AGE_GRP', 'INCOME',
                            'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4',
                            'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8',
                            'TRAVEL_MOTIVE_1', 'TRAVEL_NUM', 'TRAVEL_COMPANIONS_NUM', 'sido_gungu_list'])

for i in tqdm(list(data['TRAVEL_ID'].unique())):
    temp_df = data[data['TRAVEL_ID'] == i]
    temp_df1 = temp_df[['SIDO', 'GUNGU']] # Check which regions each user has visited
    temp_df1.reset_index(drop=True, inplace=True)
    sido_gungu_visit = []
    for j in range(len(temp_df1)):
        sido_gungu_visit.append(temp_df1['SIDO'][j] + '+' + temp_df1['GUNGU'][j])
    sido_gungu_list = list(set(sido_gungu_visit))
    new = temp_df.drop(['SIDO', 'GUNGU'], axis=1) # Remove existing region columns
    new = new.head(1)
    new['sido_gungu_list'] = str(sido_gungu_list)
    data1 = pd.concat([data1, new], axis=0) # Create new DataFrame

data1.reset_index(drop=True, inplace=True)
print("User region information processed.")

# Extract attraction information
print("Processing attraction information...")
info = filtered_train[['SIDO', 'VISIT_AREA_NM', 'GUNGU', 'VISIT_AREA_TYPE_CD', 
                      'RESIDENCE_TIME_MIN_mean', 'RCMDTN_INTENTION_mean',
                      'REVISIT_YN_mean', 'TRAVEL_COMPANIONS_NUM_mean', 'REVISIT_INTENTION_mean']]
info.drop_duplicates(['VISIT_AREA_NM'], inplace=True)

# Filter for attractions with minimum visits (5 or more) from train set
print("Filtering attractions with 5 or more visits...")
visiting_list = filtered_train[['VISIT_AREA_NM']]
visit_counts = pd.DataFrame(visiting_list.value_counts(), columns=['count'])
visit_counts['VISIT_AREA_NM'] = visit_counts.index
visit_counts.reset_index(drop=True, inplace=True)

# Data preprocessing - clean up attraction names
for i in range(len(visit_counts)):
    visit_counts['VISIT_AREA_NM'][i] = str(visit_counts['VISIT_AREA_NM'][i])
    visit_counts['VISIT_AREA_NM'][i] = visit_counts['VISIT_AREA_NM'][i].replace("(","").replace(")","").replace(",","").replace("\''","")
    visit_counts['VISIT_AREA_NM'][i] = visit_counts['VISIT_AREA_NM'][i][1:-1]

# Keep only attractions with 5 or more visits
visit_counts = visit_counts[visit_counts['count'] >= 5]
visit_list = list(visit_counts['VISIT_AREA_NM'])
print(f"Number of attractions with 5 or more visits: {len(visit_list)}")

# Filter info DataFrame for attractions with 5 or more visits
info.reset_index(drop=True, inplace=True)
indices_to_drop = []
for i in range(len(info)):
    if info['VISIT_AREA_NM'][i] not in visit_list:
        indices_to_drop.append(i)
info = info.drop(indices_to_drop, axis=0)
info.reset_index(drop=True, inplace=True)
print(f"Filtered attraction info shape: {info.shape}")

# Generate recommendations for each user
print("Generating top 10 recommendations for each user...")
result = []
for i in tqdm(range(len(data1))):
    # Create an empty DataFrame for this user's predictions
    final_df = pd.DataFrame(columns=['VISIT_AREA_NM', 'SIDO', 'GUNGU', 'VISIT_AREA_TYPE_CD',
            'TRAVEL_MISSION_PRIORITY', 'GENDER', 'AGE_GRP', 'INCOME',
            'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4',
            'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8',
            'TRAVEL_MOTIVE_1', 'TRAVEL_NUM', 'TRAVEL_COMPANIONS_NUM',
            'RESIDENCE_TIME_MIN_mean', 'RCMDTN_INTENTION_mean', 'REVISIT_YN_mean',
            'TRAVEL_COMPANIONS_NUM_mean', 'REVISIT_INTENTION_mean'])
    
    # Process the list of regions the user has visited
    temp = data1['sido_gungu_list'][i].replace("[","").replace("]","").replace("\'","").replace(", ",",")
    places_list = list(map(str, temp.split(",")))
    
    # For each region the user has visited
    for q in places_list:
        sido, gungu = map(str, q.split("+"))
        
        # Find attractions in this region
        info_df = info[(info['SIDO'] == sido) & (info['GUNGU'] == gungu)]
        if info_df.empty:
            continue
            
        info_df = info_df.copy()  # Create a copy to avoid SettingWithCopyWarning
        info_df.drop(['SIDO'], inplace=True, axis=1)
        info_df.reset_index(inplace=True, drop=True)
        
        # Get user information
        data2 = data1.drop(['sido_gungu_list'], axis=1)
        user_df = pd.DataFrame([data2.iloc[i].to_list()]*len(info_df), 
                              columns=['TRAVEL_ID', 'TRAVEL_MISSION_PRIORITY', 'GENDER', 'AGE_GRP', 'INCOME',
                                    'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4',
                                    'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8',
                                    'TRAVEL_MOTIVE_1', 'TRAVEL_NUM', 'TRAVEL_COMPANIONS_NUM'])
        
        # Combine user and attraction info
        df = pd.concat([user_df.reset_index(drop=True), info_df.reset_index(drop=True)], axis=1)
        df = df[['VISIT_AREA_NM', 'SIDO', 'GUNGU', 'VISIT_AREA_TYPE_CD',
               'TRAVEL_MISSION_PRIORITY', 'GENDER', 'AGE_GRP', 'INCOME',
               'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4',
               'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8',
               'TRAVEL_MOTIVE_1', 'TRAVEL_NUM', 'TRAVEL_COMPANIONS_NUM',
               'RESIDENCE_TIME_MIN_mean', 'RCMDTN_INTENTION_mean', 'REVISIT_YN_mean',
               'TRAVEL_COMPANIONS_NUM_mean', 'REVISIT_INTENTION_mean']]
        
        # Convert all categorical features to string type for prediction
        for col in categorical_features:
            if col in df.columns:
                df[col] = df[col].astype(str)
        
        # Add to the final DataFrame for this user
        final_df = pd.concat([final_df, df], axis=0)
    
    # Reset index and remove duplicates
    if final_df.empty:
        # If no attractions were found for this user's regions, add empty result
        rec = []
        result.append(rec)
        continue
    
    final_df.reset_index(drop=True, inplace=True)
    final_df.drop_duplicates(['VISIT_AREA_NM'], inplace=True)
    
    # Remove TRAVEL_ID for prediction (not used in the model)
    pred_df = final_df.drop(['TRAVEL_ID'], axis=1, errors='ignore')
    
    # Prepare prediction data - make sure it has the same format as training data
    for col in categorical_features:
        if col in pred_df.columns:
            pred_df[col] = pred_df[col].astype(str)
    
    # Ensure all columns needed for prediction are present
    for col in X_train.columns:
        if col not in pred_df.columns:
            # If a column is missing, create it with default values
            if col in categorical_features:
                pred_df[col] = "unknown"
            else:
                pred_df[col] = 0
    
    # Keep only columns used during training and in the same order
    pred_df = pred_df[X_train.columns]
    
    try:
        # Model prediction
        y_pred = model.predict(pred_df)
    except Exception as e:
        print(f"Error during prediction for user {i}: {str(e)}")
        # Create empty predictions if there's an error
        y_pred = np.zeros(len(pred_df))
    y_pred = pd.DataFrame(y_pred, columns=['y_pred'])
    test_df1 = pd.concat([final_df, y_pred], axis=1)
    test_df1.sort_values(by=['y_pred'], axis=0, ascending=False, inplace=True)  # Sort by prediction score
    
    # Select top 10 attractions
    test_df1 = test_df1.iloc[0:10,]
    
    # Get list of recommended attractions
    visiting_candidates = list(test_df1['VISIT_AREA_NM'])
    
    # Create result record with user info and recommendations
    test_df2 = test_df1[['SIDO', 'TRAVEL_MISSION_PRIORITY', 'GENDER', 'AGE_GRP', 'INCOME',
                        'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4',
                        'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8',
                        'TRAVEL_MOTIVE_1', 'TRAVEL_NUM', 'TRAVEL_COMPANIONS_NUM']]
    
    if len(test_df2) == 0:
        rec = []
        result.append(rec)
    else:
        rec = test_df2.iloc[0].to_list()
        rec.append(visiting_candidates)
        result.append(rec)

# Create results DataFrame with recommendations
final_df = pd.DataFrame(result,
                      columns=['SIDO', 'TRAVEL_MISSION_PRIORITY', 'GENDER', 'AGE_GRP', 'INCOME',
                               'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4',
                               'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8',
                               'TRAVEL_MOTIVE_1', 'TRAVEL_NUM', 'TRAVEL_COMPANIONS_NUM', 
                               'recommend_result_place'])

# Keep only recommendations column and add TRAVEL_ID
final_df = final_df[['recommend_result_place']]
travel_id = data1[['TRAVEL_ID']]
travel_id.reset_index(drop=True, inplace=True)
final_df = pd.concat([travel_id, final_df], axis=1)

# Save recommendations
output_path = f'catboost_recommendations_{timestamp}.csv'
final_df.to_csv(output_path, index=False)
print(f"Recommendations saved to {output_path}")

# Calculate Recall@10
print("Calculating Recall@10 for recommendations...")

# Identify users with fewer than 10 recommendations
travel_id_list = []
for i in range(len(final_df)):
    recommend_list = final_df['recommend_result_place'][i]
    if pd.isna(recommend_list):
        travel_id_list.append(final_df['TRAVEL_ID'][i])
        continue
    if str(recommend_list).count(',') < 9:
        travel_id_list.append(final_df['TRAVEL_ID'][i])

# Extract list of region names (without -구, -시, -군 suffixes)
places = list(set(test_with_extended_stats['GUNGU']))
for i in range(len(places)):
    if places[i][-1] == '구' or places[i][-1] == '시' or places[i][-1] == '군':
        places[i] = places[i][:-1]

# Calculate Recall@10 for each user
recall_10_list = []
for i in tqdm(list(test_with_extended_stats['TRAVEL_ID'].unique())):
    # If user has fewer than 10 recommendations, set recall to 0
    if i in travel_id_list:
        recall_10_list.append(0)
        continue
    
    # Get attractions this user was satisfied with (rating >= 4)
    satisfied = test_with_extended_stats[test_with_extended_stats['TRAVEL_ID'] == i]
    satisfied.reset_index(drop=True, inplace=True)
    
    satisfied1 = satisfied[satisfied['DGSTFN'] >= 4]
    if len(satisfied1) == 0:  # If user wasn't satisfied with any attractions
        recall_10_list.append(0)
        continue
    
    item_list = list(set(satisfied1['VISIT_AREA_NM']))
    
    # Get the model's recommendations for this user
    recommend_list = final_df[final_df['TRAVEL_ID'] == i]['recommend_result_place']
    
    # Count matches between satisfied attractions and recommendations
    summ = 0
    for n in item_list:
        word_list = list(n.split(' '))
        if word_list[-1][-1] == '점':  # Remove location suffix (e.g., "강남점")
            del word_list[-1]
        for o in word_list:
            if o in places:  # Skip region names
                pass
            else:
                for p in recommend_list:  # Check if word is in recommended place name
                    if o in str(p):
                        summ += 1
    
    # Calculate recall@10
    recall10_for_1user = summ / min(10, len(satisfied1))
    if recall10_for_1user > 1:
        recall10_for_1user = 1
    recall_10_list.append(recall10_for_1user)

# Calculate and display overall Recall@10
recall_10 = np.mean(recall_10_list)
print(f"Recall@10: {recall_10:.4f}")

# Save Recall@10 results
recall_df = pd.DataFrame({
    'TRAVEL_ID': list(test_with_extended_stats['TRAVEL_ID'].unique()),
    'Recall@10': recall_10_list
})
recall_df.to_csv(f'recall_at_10_results_{timestamp}.csv', index=False)
print(f"Recall@10 results saved to recall_at_10_results_{timestamp}.csv")

print("CatBoost recommendation process completed!")

Loading preprocessed datasets...
Train set shape: (614294, 36)
Test set shape: (186295, 161)
Place stats shape: (11467, 126)
Preparing data for CatBoost model...
Identifying and converting categorical features...
Identified 24 categorical features:
['MVMN_SE_NM', 'ACTIVITY_TYPE_CD', 'STORE_NM', 'REL_CD', 'VISIT_AREA_NM', 'SIDO', 'GUNGU', 'VISIT_AREA_TYPE_CD', 'TRAVEL_MISSION_PRIORITY', 'GENDER', 'AGE_GRP', 'INCOME', 'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8', 'TRAVEL_MOTIVE_1', 'TRAVEL_NUM', 'TRAVEL_COMPANIONS_NUM', 'RESIDENCE_TIME_MIN_mean']
Total features: 29
Training CatBoost model...
Training CatBoost model...
Learning rate set to 0.100785
0:	learn: 0.7577307	total: 919ms	remaining: 17m 35s


KeyboardInterrupt: 